In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('jumia.csv.xls')
data.head()

,brand,current_price,old_price,discount,description,ratings,times_reviewed
0,Gionee,"₦ 48,000","₦ 73,630",-35%,Gionee K3 (M100) 4GB+64GB 6.22 Inch HD+ Androi...,70,10
1,Nokia,"₦ 59,520","₦ 119,660",-50%,"7， 5.2-inch (4GB, 64GB ROM) Android 7.1, 16MP ...",76,10
2,UMIDIGI,"₦ 25,990","₦ 29,990",-13%,"A3S Android 10 Global Band 3950mAh 5.7"" Smartp...",86,99
3,Nokia,"₦ 42,990","₦ 70,485",-39%,3.2 3GB RAM 32GB 6.26 Inch 13MP Camera Dual SI...,81,104
4,UMIDIGI,"₦ 25,990","₦ 29,990",-13%,"A3S Android 10 Global Band 3950mAh 5.7"" Smartp...",78,148


In [3]:
# checking for duplicate rows
data[data.duplicated()]

,brand,current_price,old_price,discount,description,ratings,times_reviewed


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 7 columns):
brand             502 non-null object
current_price     508 non-null object
old_price         350 non-null object
discount          508 non-null object
description       508 non-null object
ratings           508 non-null int64
times_reviewed    508 non-null int64
dtypes: int64(2), object(5)
memory usage: 27.9+ KB


In [5]:
# cleaning the current price column
data['current_price'] = data['current_price'].str[1:].str.replace(',','').astype('int')

# cleaning the old price column
data['old_price'] = data['old_price'].str.replace('[₦\s,]', '')
data['old_price'] = data['old_price'].fillna(0).astype('int')

# cleaning the discount column
data['discount'] = data['discount'].str.strip('[% ₦]') 
data['discount'].loc[data['discount'].apply(lambda x: len(str(x)) > 4)] = 0

# cleaning the ratings column
data.ratings = data.ratings.apply(lambda x: 5*x/100)

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## Method 1

Leverages on the pandas regular expression methods (extractall) to find matching patterns, then those matching patterns will be returned as a series which is then used to create a column within the dataframe

In [6]:
# Creating a function that will help the data cleaning task
def clean_description(series, col):
    """
    clean_description(series, col)
    
    This function takes in a series and creates a column for that series within the an already existing dataframe. 
    It takes in two parameters:
    
    Parameters
    ----------
    series: pd.Series
        An already existing series object
    col: str
        Column name for the series object
    
    Returns
    ------
    Dataframe
    """
    # creating the column
    data[col] = np.NaN 
    
    # selecting the subset of our dataframe that are present in the series argument
    subset = data.loc[series.index] 
    
    subset[col] = series # fitting the subset of dataframe with values from the series
    data.loc[series.index] = subset # replacing the portion of the dataframe with our subset

## Method 2

Using the re library to match patterns that automatically return a dataframe that can then be concatenated to the existing dataframe. 

In [7]:
def clean(pattern, cols, names):
    import re
    """
    clean(pattern, cols, names)
    
    This function takes a pattern and returns a series or dataframe (depending on the way it matches). It takes in 
    three parameters:
        
    Parameters
    ----------
    pattern : str
        A regular expression that matches one or more options. 
            
    col : list
        A list that selects out the matched dataframe 
        
    names : list
        A list of strings that will replace the name of the columns within the dataframe
        
    Returns
    -------
    DataFrame
    """
    match = data['description'].apply(lambda x: re.findall(pattern, x))
    match_df = match.apply(lambda x: ','.join(x))
    match_df.columns = names
    return match_df.apply(lambda x: x.replace(['', None], np.NaN))

### Extracting the Rom and Ram

In [8]:
# Extracting the rom and ram
ram_rom = data['description'].str.extractall('(\d+GB)')

# selecting the rom from ram_rom
rom = ram_rom.xs(1, level = 'match')[0].str.replace('GB', '').astype('int')
clean_description(rom, 'rom') # creating the rom column

# selecting the ram from ram_rom
ram = ram_rom.xs(0, level = 'match')[0].str.replace('GB', '').astype('int')
clean_description(ram, 'ram') # creating the ram column

### Cleaning the Rom and Ram Columns

In [9]:
# Indexing out the data points where ram is bigger than rom 
mismatched = data[data['rom'] < data['ram']]
mismatched.rename(columns = {"ram":'rom', 'rom':'ram'}, inplace = True) # renaming the column to correct the abnormal

# rematching the values back to the dataframe
data.loc[mismatched.index] = mismatched

# the ram of a phone is rarely above 12
abnormal_ram = data[data.ram > 12]

# indexing out abnormal matches for the ram_rom column
ram_ = data['description'].str.extractall('([\d\+]+[GM]B)|([\d]+R[oaOA][mM])').xs(0, level = 'match')[0]
rom_ = data['description'].str.extractall('([\d\+]+[GM]B)|([\d]+R[oaOA][mM])').xs(1, level = 'match')

# getting the index of the mismatch
print('The index of the abnormal value within the rom_ series is:', 
      rom_[1][rom_[1] == '3RAM'].index[0])

# extracting values within the series ram_ that have unusual output
print('The index of the abnormal value within the ram_ series are:', 
      ram_.str.extractall('(\d\+\d*)').xs(0, level='match').index)

The index of the abnormal value within the rom_ series is: 114
The index of the abnormal value within the ram_ series are: Int64Index([22, 134, 161, 362, 440], dtype='int64')


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [10]:
# cleaning index 362 and 440
abnormal_ram['rom'].copy()[[362, 440, 114]] = 3

# cleaning index 22 and 161
abnormal_ram['rom'].copy()[[22, 161]] = 4

# cleaning index 134
abnormal_ram['rom'].copy()[134] = 2

# cleaning index 382
abnormal_ram['rom'].copy()[382] = 0.008

# cleaning index 482
abnormal_ram['rom'].copy()[482] = 0.542

# renaming the column to correct the abnormal
abnormal_ram.rename(columns = {"ram":'rom', 'rom':'ram'}, inplace = True) 

# rematching the values back to the dataframe
data.loc[abnormal_ram.index] = abnormal_ram

### Extracting the Battery, Camera quality, Sim, Screen dimension and O.S. Versions

In [11]:
# selecting out values that match the mah values of battery
battery = data['description'].str.extractall('([\d,]*[\s]*[Mm][aA][hH])').xs(0, level = 'match')[0].str.replace('([Mm][aA][hH])', '').str.replace(',', '').astype(float)

clean_description(battery, 'battery') # creating the battery column

In [12]:
# extracting the megapixel of phones
camera = data['description'].str.extractall('([\d.]*MP)').xs(0, level = 'match')[0].str.replace('MP','').str.strip('.').replace('', np.NaN).astype(float)

clean_description(camera, 'camera') # creating the battery column

In [13]:
# extracting the sim ports of phones
sims = data['description'].str.extractall('([\w]*[\s]*[Ss][iI][Mm])|([\w]*[\s]*[Ss]lots)').xs(0, level='match')
sims = sims[0].combine_first(sims[1])

clean_description(sims, 'sim port') # creating the sim port column

In [14]:
# extracting the screen dimension of phones
screen = data['description'].str.extractall('(\d*\.\d*[\'"])|([\d.\'"-]*[\s]*[Ii][nN][Cc][Hh])').xs(0, level='match')
screen = screen[0].combine_first(screen[1])
screen = screen.str.replace('[\'"iInch-]', '').astype(float)
clean_description(screen, 'screen_dim') # creating the screen dimension column

In [15]:
android = data['description'].str.extractall("([Aa]ndr[oi]*d [\s\d.-]{,3})").xs(0, level = 'match')[0].str.replace('[Aa]ndr[oi]*d[\s-]*', '').str.strip('.').replace('', np.NaN).astype(float)

clean_description(android, 'android_ver') # creating the Android version column

In [16]:
data.isnull().sum()

brand               6
current_price       0
old_price           0
discount            0
description         0
ratings             0
times_reviewed      0
rom                46
ram                52
battery           292
camera            177
sim port          303
screen_dim         67
android_ver       227
dtype: int64

### Cleaning Null Values

In [17]:
data['brand'].fillna(data['brand'].mode()[0], inplace = True)
data['rom'].fillna(data['rom'].median(), inplace = True)
data['ram'].fillna(data['ram'].median(), inplace = True) # mode and median are the same
data['battery'].fillna(data['battery'].median(), inplace = True)
data['camera'].fillna(data['camera'].median(), inplace = True) # median and mean are the same
data['sim port'].fillna(data['sim port'].mode()[0], inplace = True) # mode is for filling categorical data
data['screen_dim'].fillna(data['screen_dim'].median(), inplace = True) # median and mean are the same
data['android_ver'].fillna(data['android_ver'].median(), inplace = True)

In [18]:
data.isnull().sum()

brand             0
current_price     0
old_price         0
discount          0
description       0
ratings           0
times_reviewed    0
rom               0
ram               0
battery           0
camera            0
sim port          0
screen_dim        0
android_ver       0
dtype: int64

In [19]:
data.to_csv('Jumia_clean.csv', index = False)